# Implementation of Softmax Regression from Scratch

Just as we implemented linear regression from scratch, we believe that multiclass logistic (softmax) regression
is similarly fundamental and you ought to know the gory details of how to implement it from scratch. As
with linear regression, after doing things by hand we will breeze through an implementation in PyTorch for
comparison. To begin, let’s import our packages

In [ ]:
# import packages
import torch
import torchvision
import numpy as np
import d2l
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
from IPython import display

We will work with the Fashion-MNIST dataset just introduced, cuing up an iterator with batch size 256.

In [ ]:
def load_data_fashion_mnist(batch_size, resize=None, num_workers=0):
    tranform_list = []
    if resize:
        tranform_list.append(torchvision.transforms.Resize(resize))
    tranform_list.append(transforms.ToTensor())
    transform = transforms.Compose(tranform_list)
    mnist_train = datasets.FashionMNIST('~/datasets/F_MNIST/',
                                 download=True,
                                 train=True,
                                 transform=transform)

    mnist_test = datasets.FashionMNIST('~/datasets/F_MNIST/',
                                     download=True,
                                     train=False,
                                    transform=transform)
    
    train_iter = torch.utils.data.DataLoader(mnist_train, batch_size=batch_size, 
                                          shuffle=True, num_workers=num_workers)
    test_iter = torch.utils.data.DataLoader(mnist_test, batch_size=batch_size, 
                                              shuffle=False)
    return train_iter, test_iter

In [ ]:
batch_size = 256
train_iter, test_iter = # insert your code here

## Initialize Model Parameters

Just as in linear regression, we represent each example as a vector. Since each example is a 28 x 28 image,
we can flatten each example, treating them as 784 dimensional vectors. In the future, we’ll talk about more
sophisticated strategies for exploiting the spatial structure in images, but for now we treat each pixel location
as just another feature.

Recall that in softmax regression, we have as many outputs as there are categories. Because our dataset has
10 categories, our network will have an output dimension of 10. Consequently, our weights will constitute a
784 x 10 matrix and the biases will constitute a 1 x 10 vector. As with linear regression, we will initialize
our weights W with Gaussian noise and our biases to take the initial value 0.

In [ ]:
num_inputs = # insert your code here
num_outputs = # insert your code here

W = # insert your code here
b = # insert your code here

Recall that we need to attach gradients to the model parameters. More literally, we are allocating memory
for future gradients to be stored and notifiying PyTorch that we want gradients to be calculated with respect
to these parameters in the first place.

In [ ]:
# insert your code here

## The Softmax

Before implementing the softmax regression model, let’s briefly review how operators such as sum work along
specific dimensions in a tensor. Given a matrix X we can sum over all elements (default) or only over
elements in the same column (axis=0) or the same row (axis=1). 

Note that if X is an array with shape (2, 3) and we sum over the columns (X.sum(axis=0), the result will be a (1D) vector with shape (3,). If we
want to keep the number of axes in the original array (resulting in a 2D array with shape (1,3)), rather
than collapsing out the dimension that we summed over we can specify keepdims=True when invoking sum.

In [ ]:
X = torch.tensor([[1, 2, 3], [4, 5, 6]])

In [ ]:
# illustrate using dim=0

In [ ]:
# illustrate using dim=1

In [ ]:
# illustrate using dim=1 and keepdims

We are now ready to implement the softmax function. Recall that softmax consists of two steps: First, we
exponentiate each term (using exp). Then, we sum over each row (we have one row per example in the
batch) to get the normalization constants for each example. Finally, we divide each row by its normalization
constant, ensuring that the result sums to 1. Before looking at the code, let’s recall what this looks expressed
as an equation: $$\mbox{softmax}(\mathbf{X})_{ij} = \frac{\exp(X_{ij})}{\sum_k \exp(X_{ik})}$$.

In [ ]:
def softmax(X):
    # insert your code here

In [ ]:
# illustrate using 2 x 3 matrix and show columns sum up to 1

## The Model

Now that we have defined the softmax operation, we can implement the softmax regression model. The
below code defines the forward pass through the network. Note that we flatten each original image in the
batch into a vector with length num_inputs with the reshape function before passing the data through our
model.

In [ ]:
def net(X):
    # insert your code here

In [ ]:
# illustrate net with 10, 28, 28 random vector

## The Loss Function

Next, we need to implement the cross-entropy loss function, introduced in the class. This may be the most
common loss function in all of deep learning because, at the moment, classification problems far outnumber
regression problems.

Recall that cross-entropy takes the negative log likelihood of the predicted probability assigned to the true
label $\log p(y|x)$.

In [ ]:
def cross_entropy(y_hat, y):
    # insert your code here

## Classification Accuracy

Given the predicted probability distribution ``y_hat``, we typically choose the class with highest predicted
probability whenever we must output a hard prediction.

In [ ]:
def accuracy(y_hat, y):
    # insert your code here

Here, we define ``Accumulator`` is a utility class to accumulate sum over multiple numbers.

In [ ]:
class Accumulator(object):
    """Sum a list of numbers over time"""
    def __init__(self, n):
        self.data = [0.0] * n
    def add(self, *args):
        self.data = [a+b for a, b in zip(self.data, args)]
    def reset(self):
        self.data = [0] * len(self.data)
    def __getitem__(self, i):
        return self.data[i]

Now, we can evaluate the accuracy for model net on the data set (accessed via data_iter).

In [ ]:
def evaluate_accuracy(net, data_iter):
    metric = Accumulator(2)
    for X, y in data_iter:
        metric.add(accuracy(net(X), y), len(y))
    return float(metric[0])/metric[1]

In [ ]:
# illustrate evaluate_accuracy on our test_iter and net

## Model Training

The training loop for softmax regression should look strikingly familiar to our implementation
of linear regression. Here we refactor the implementation to make it reusable. First,
we define a function to train for one data epoch. Note that updater is general function to update the model
parameters. 

Let's grab our sgd function we used in linear regression from scratch notebook.

In [ ]:
def sgd(params, lr):
    # insert your code here

In [ ]:
def train_epoch(net, train_iter, loss, updater):
    metric = Accumulator(3) # train_loss_sum, train_acc_sum, num_examples
    # insert your code here

Before showing the implementation of the training function, we define a utility class that draw data in
animation. Again, it aims to simplify the codes in later chapters.

In [ ]:
class Animator(object):
    def __init__(self, xlabel=None, ylabel=None, legend=[], xlim=None,
                 ylim=None, xscale='linear', yscale='linear', fmts=None,
                 nrows=1, ncols=1, figsize=(3.5, 2.5)):
        """Incrementally plot multiple lines."""
        d2l.use_svg_display()
        self.fig, self.axes = d2l.plt.subplots(nrows, ncols, figsize=figsize)
        if nrows * ncols == 1: self.axes = [self.axes,]
        # use a lambda to capture arguments
        self.config_axes = lambda : d2l.set_axes(self.axes[0], xlabel, ylabel,
                                                 xlim, ylim, xscale, yscale, legend)
        self.X, self.Y, self.fmts = None, None, fmts
    def add(self, x, y):
        """Add multiple data points into the figure."""
        if not hasattr(y, "__len__"): y = [y]
        n = len(y)
        if not hasattr(x, "__len__"): x = [x] * n
        if not self.X: self.X = [[] for _ in range(n)]
        if not self.Y: self.Y = [[] for _ in range(n)]
        if not self.fmts: self.fmts = ['-'] * n
        for i, (a, b) in enumerate(zip(x, y)):
            if a is not None and b is not None:
                self.X[i].append(a)
                self.Y[i].append(b)
        self.axes[0].cla()
        for x, y, fmt in zip(self.X, self.Y, self.fmts):
            self.axes[0].plot(x, y, fmt)
        self.config_axes()
        display.display(self.fig)
        display.clear_output(wait=True)

The training function then runs multiple epochs and visualize the training progress.

In [ ]:
def train(net, train_iter, test_iter, loss, num_epochs, updater):
    trains, test_accs = [], []
    animator = Animator(xlabel='epoch', xlim=[1, num_epochs], 
                        ylim=[0.3, 0.9], legend=['train loss', 'train acc', 'test acc'])
    for # insert your code here
        train_metrics = # insert your code here
        test_acc = # insert your code here
        animator.add(epoch+1, train_metrics+(test_acc,))

Again, we use the mini-batch stochastic gradient descent to optimize the loss function of the model. Note
that the number of epochs (num_epochs), and learning rate (lr) are both adjustable hyper-parameters. By
changing their values, we may be able to increase the classification accuracy of the model. In practice we’ll
want to split our data three ways into training, validation, and test data, using the validation data to choose
the best values of our hyperparameters.

In [ ]:
num_epochs = 10
lr = 0.1
updater = lambda: sgd([W, b], lr)

train(net, train_iter, test_iter, cross_entropy, num_epochs, updater)


## Prediction

Now that training is complete, our model is ready to classify some images. Given a series of images, we will
compare their actual labels (first line of text output) and the model predictions (second line of text output).

Let's grab our functions for visualization and Fashion-MNIST labels from the notebook ``image_classification_data_Fashion-MNIST``.

In [ ]:
# taken from d2l but modified to fit to PyTorch
def show_images(imgs, num_rows, num_cols, titles=None, scale=1.5):
    """Plot a list of images."""
    figsize = (num_cols * scale, num_rows * scale)
    _, axes = d2l.plt.subplots(num_rows, num_cols, figsize=figsize)
    axes = axes.flatten()
    for i, (ax, img) in enumerate(zip(axes, imgs)):
        ax.imshow(img.squeeze(0).numpy())
        ax.axes.get_xaxis().set_visible(False)
        ax.axes.get_yaxis().set_visible(False)
        if titles:
            ax.set_title(titles[i])
            
def get_fashion_mnist_labels(labels):
    text_labels = ['t-shirt', 'trouser', 'pullover', 'dress', 'coat',
                  'sandal', 'shirt', 'sneaker', 'bag', 'ankle_boot']
    return [text_labels[int(i)] for i in labels]

In [ ]:
def predict(net, test_iter, n=6):
    for X, y in test_iter:
        break
    trues = get_fashion_mnist_labels(y.numpy())
    preds = get_fashion_mnist_labels(net(X).argmax(axis=1).numpy())
    titles = [true+'\n'+ pred for true, pred in zip(trues, preds)]
    show_images(X[0:n].reshape((n,28,28)), 1, n, titles=titles[0:n])

In [ ]:
predict(net, test_iter, 16)

## Summary

With softmax regression, we can train models for multi-category classification. The training loop is very
similar to that in linear regression: retrieve and read data, define models and loss functions, then train
models using optimization algorithms. As you’ll soon find out, most common deep learning models have
similar training procedures.